# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116114e+02     1.475838e+00
 * time: 0.7818489074707031
     1     1.061991e+01     8.551550e-01
 * time: 2.0250279903411865
     2    -1.222648e+01     9.711463e-01
 * time: 2.656803846359253
     3    -3.426443e+01     7.090390e-01
 * time: 3.5254788398742676
     4    -4.796473e+01     5.255754e-01
 * time: 4.40988302230835
     5    -5.707416e+01     1.880500e-01
 * time: 5.308797836303711
     6    -5.988715e+01     1.123665e-01
 * time: 5.891636848449707
     7    -6.091234e+01     4.251702e-02
 * time: 6.475327014923096
     8    -6.129384e+01     7.292531e-02
 * time: 7.059955835342407
     9    -6.159560e+01     3.067346e-02
 * time: 7.664609909057617
    10    -6.179642e+01     2.587511e-02
 * time: 8.252810001373291
    11    -6.195088e+01     1.819679e-02
 * time: 8.837464809417725
    12    -6.201352e+01     1.556165e-02
 * time: 9.422030925750732
    13    -6.208435e+01     1.549268e-02
 * time: 10.007568836212158
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671060
    AtomicLocal         -18.8557697
    AtomicNonlocal      14.8522675
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336818

    total               -62.261666455791